# Segmenting and Clustering Neighborhoods in Toronto #

## Question 1 ##

- **Build a dataframe of the postal code of each neighborhood with borough name and neighborhood name in Toronto.**

**Importing librairies**

In [307]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

**Scraping data from Wikipedia page into a Dataframe**

In [308]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_text = requests.get(url).text
soup = BeautifulSoup(html_text, 'html.parser')

In [309]:
#Creating empty dataframe with columns
column_names = ['PostalCode','Borough','Neighborhood']
toronto_neighbours = pd.DataFrame(columns = column_names)

postal_code = ""
borough = ""
neighborhood = ""

# Get the first table which contains all the postcode, borough, neighborhood information
table = wiki_data.find('table')

# Put the table information in the dataframe
for td in table.findAll('td'):
    postal_code = td.b.text
    text = td.span.text
    #If there is a neighbourhood, then the dataframe is filled
    if(text != 'Not assigned'):
        borough = text.split('(')[0]
        neighborhood = ((((text.split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        toronto_neighbours = toronto_neighbours.append({'PostalCode': postal_code,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

#We replace some borough values with other borough values    
toronto_neighbours['Borough']=toronto_neighbours['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
toronto_neighbours.head()



,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [310]:
print("Dataframe shape: {}".format(toronto_neighbours.shape))

Dataframe shape: (103, 3)


## Question 2 ##

- **Get the latitude and the longitude coordinates of each neighborhood and include them to the previous dataframe**

**Installing geocoder library**

In [311]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


**Importing geocoder library**

In [312]:
import geocoder

**Importing geospatial coordinates from csv file**

In [313]:
geospatial_coords = pd.read_csv('Geospatial_Coordinates.csv')
geospatial_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Joining geospatial coordinates with dataframe composed of postal codes, borough and neighborhood info**

In [314]:
merged_data = toronto_neighbours.join(geospatial_coords.set_index('Postal Code'), on='PostalCode')
merged_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [315]:
merged_data.shape

(103, 5)

**Verification that the dataframe is like the required dataframe**

In [316]:
post_codes_test = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L',
                   'M5V', 'M1B', 'M5A']


column_names_test = ['PostalCode','Borough','Neighborhood', 'Latitude', 'Longitude']
test_df = pd.DataFrame(columns = column_names_test)

for postcode in post_codes_test:
    test_df = test_df.append(merged_data[merged_data["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


## Question 3 ##

- **Explore and cluster the neighborhoods in Toronto**

**Installing geopy and folium libraries**

In [317]:
!pip install geopy
!pip install folium

**Importing librairies**

In [318]:
from geopy.geocoders import Nominatim
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

**Using geopy library to get the latitude and longitude values of Toronto**

In [319]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


**Creating a map of Toronto with neighbourhoods**

In [320]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for latitude, longitude, borough, neighborhood in zip(merged_data['Latitude'], merged_data['Longitude'], merged_data['Borough'], merged_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_toronto)  
    
map_toronto

**Filtering boroughs that contain the word Toronto**

In [321]:
toronto_df = merged_data[merged_data.Borough.str.contains("Toronto")].reset_index(drop=True)
print(toronto_df.shape)
toronto_df.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


**Showing map of Toronto with neighborhood in boroughs that contain the word Toronto**

In [322]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# adding markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_toronto)  
map_toronto

**Credentials for the Foursquare API to explore the neighborhoods**

In [323]:
CLIENT_ID = 'ZF1TM4YOQHTNZTPQMRJJOS2VMWLAGYXUNWOGKFYBFLNGJHUI' 
CLIENT_SECRET = '3SILTMMAWVOVQ0P4PTFWXALNFQJ52KHHYPQ2AZZNCTWONJ20'
VERSION = '20180605'
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZF1TM4YOQHTNZTPQMRJJOS2VMWLAGYXUNWOGKFYBFLNGJHUI
CLIENT_SECRET:3SILTMMAWVOVQ0P4PTFWXALNFQJ52KHHYPQ2AZZNCTWONJ20


**Creating a function to get all venues that are within a radius of 500 meters for each neighborhood**

In [324]:
def getNearbyVenues(posts, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for post, name, lat, lng in zip(posts, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return relevant information for each venue
        venues_list.append([(
            post,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode',
                             'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [325]:
venues_toronto = getNearbyVenues(merged_data['PostalCode'], merged_data['Neighborhood'], merged_data['Latitude'], merged_data['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [326]:
venues_toronto.shape

(2112, 8)

In [327]:
venues_toronto.head()

,PostalCode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


**Showing the venues based on Neighborhood**

In [328]:
venues_toronto.groupby('Neighborhood').head()

,PostalCode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
...,...,...,...,...,...,...,...,...
2097,M8Z,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Wingporium,43.630275,-79.518169,Wings Joint
2098,M8Z,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,South St. Burger,43.631314,-79.518408,Burger Joint
2099,M8Z,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Dollarama,43.629883,-79.518627,Discount Store
2100,M8Z,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Healthy Planet,43.630214,-79.518495,Supplement Shop


**Showing the number of unique venue categories**

In [330]:
print('There are {} uniques categories.'.format(len(venues_toronto['Venue Category'].unique())))
venues_toronto['Venue Category'].unique()[:20]

There are 271 uniques categories.


array(['Park', 'Fast Food Restaurant', 'Food & Drink Shop',
       'Hockey Arena', 'Coffee Shop', 'Portuguese Restaurant',
       'Intersection', 'Financial or Legal Service', 'Bakery',
       'Distribution Center', 'Restaurant', 'Spa', 'Breakfast Spot',
       'Gym / Fitness Center', 'Historic Site', 'Chocolate Shop',
       'Farmers Market', 'Pub', 'Dessert Shop', 'Performing Arts Venue'],
      dtype=object)

**Analyse each neighborhood**

In [331]:
# One-hot encoding
venues_toronto_onehot = pd.get_dummies(venues_toronto[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
venues_toronto_onehot['Neighborhood'] = venues_toronto['Neighborhood'] 

# Move neighborhood column to the first column
fixed_columns = [venues_toronto_onehot.columns[-1]] + list(venues_toronto_onehot.columns[:-1])
venues_toronto_onehot = venues_toronto_onehot[fixed_columns]

venues_toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [332]:
venues_toronto_onehot.shape

(2112, 271)

**We group rows by neighborhood and we take the mean of the frequency of occurrence of each category**

In [333]:
venues_toronto_grouped = venues_toronto_onehot.groupby('Neighborhood').mean().reset_index()
venues_toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
96,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0


In [334]:
venues_toronto_grouped.shape

(100, 271)

**Let's print each neighborhood along with the top 5 most common venues**

In [335]:
num_top_venues = 5

for venue in venues_toronto_grouped['Neighborhood']:
    print("----"+ venue +"----")
    temp = venues_toronto_grouped[venues_toronto_grouped['Neighborhood'] == venue].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4         Mexican Restaurant  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1        Pharmacy  0.11
2            Pool  0.11
3  Sandwich Place  0.11
4             Pub  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0                 Bank  0.09
1          Coffee Shop  0.09
2          Bridal Shop  0.04
3            Gift Shop  0.04
4  Fried Chicken Joint  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4        Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0              Coffee Shop  0.08
1               Restaurant  0.08
2           Sandwich

4        Coffee Shop   0.2


----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                           Café  0.08
1                    Coffee Shop  0.07
2  Vegetarian / Vegan Restaurant  0.05
3          Vietnamese Restaurant  0.05
4            Arts & Crafts Store  0.03


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                             venue  freq
0                   Sandwich Place   0.5
1                             Park   0.5
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Lawrence Manor, Lawrence Heights----
                    venue  freq
0          Clothing Store  0.33
1  Furniture / Home Store  0.17
2           Women's Store  0.08
3   Vietnamese Restaurant  0.08
4      Miscellaneous Shop  0.08


----Lawrence Park----
                       venue  freq
0                   Bus Line  0.33
1                       Park  

4        Monument / Landmark   0.0


----Wexford, Maryvale----
                       venue  freq
0          Accessories Store  0.12
1  Middle Eastern Restaurant  0.12
2                Auto Garage  0.12
3                     Bakery  0.12
4             Sandwich Place  0.12


----Willowdale South----
              venue  freq
0  Ramen Restaurant  0.09
1       Coffee Shop  0.06
2       Pizza Place  0.06
3     Shopping Mall  0.06
4              Café  0.06


----Willowdale West----
           venue  freq
0       Pharmacy  0.17
1  Grocery Store  0.17
2    Supermarket  0.17
3    Coffee Shop  0.17
4        Butcher  0.17


----Willowdale, Newtonbrook----
                             venue  freq
0                             Park   1.0
1                      Yoga Studio   0.0
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Woburn----
                   venue  freq
0            Coffee Shop  0.67
1  Korean BBQ Restaur

**We write a function to sort the venues in descending order**

In [336]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**We create a DataFrame that contains the top 10 venues for each neighborhood**

In [337]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)

# Create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_toronto_grouped['Neighborhood']

for ind in np.arange(venues_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Playground,Pool,Pub,Gym,Airport Service,Falafel Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Sushi Restaurant,Shopping Mall,Bridal Shop,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Park
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Sushi Restaurant,Pharmacy,Pizza Place,Pub,Café,Butcher


**Importing librairies for clustering**

In [338]:
from sklearn.cluster import KMeans
import sklearn.cluster.k_means_

**Let's run k-means to cluster the neighborhood into 5 clusters.**

In [339]:
knum_clusters = 5
toronto_grouped_clustering = venues_toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=knum_clusters, random_state=1).fit(toronto_grouped_clustering)
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[1 0 1 1 1 1 1 1 1 3]
100


**Create a complete dataframe that includes the cluster, postal code, borough, neighborhood and the top 10 venues for each neighborhood**

In [340]:
# Add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = merged_data
# Join the 2 dataframes so that the latitudes and longitudes are added to the dataframe containing the clusters 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Fast Food Restaurant,Park,Food & Drink Shop,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Hockey Arena,Coffee Shop,Financial or Legal Service,Portuguese Restaurant,Intersection,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Bakery,Park,Breakfast Spot,Restaurant,Café,Pub,Theater,Spa,Beer Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Miscellaneous Shop,Coffee Shop,Women's Store,Vietnamese Restaurant,Electronics Store,Escape Room
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Burrito Place,Bar,Italian Restaurant,Japanese Restaurant,Beer Bar,Sandwich Place,Salad Place


**Let's visualize the resulting clusters**

In [341]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(knum_clusters)
ys = [i+x+(i*x)**2 for i in range(knum_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
# adding markers to map
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters

**Cluster 1**

In [342]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,0.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Dim Sum Restaurant
8,East York,0.0,Pizza Place,Pharmacy,Athletics & Sports,Gastropub,Furniture / Home Store,Intersection,Flea Market,Pet Store,Breakfast Spot,Bank
10,North York,0.0,Pizza Place,Asian Restaurant,Bakery,Japanese Restaurant,Women's Store,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
32,Scarborough,0.0,Pizza Place,Playground,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
46,North York,0.0,Grocery Store,Park,Hotel,Bank,Shopping Mall,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
50,North York,0.0,Pizza Place,Intersection,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farmers Market
60,North York,0.0,Gym / Fitness Center,Liquor Store,Discount Store,Athletics & Sports,Grocery Store,Comic Shop,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant
63,York,0.0,Brewery,Grocery Store,Bus Line,Convenience Store,Women's Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
70,Etobicoke,0.0,Intersection,Pizza Place,Discount Store,Sandwich Place,Coffee Shop,Middle Eastern Restaurant,Chinese Restaurant,Women's Store,Dog Run,Distribution Center
72,North York,0.0,Pharmacy,Supermarket,Pizza Place,Coffee Shop,Butcher,Grocery Store,Airport Food Court,Farmers Market,Event Space,Ethiopian Restaurant


**Cluster 2**

In [343]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1.0,Hockey Arena,Coffee Shop,Financial or Legal Service,Portuguese Restaurant,Intersection,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,Downtown Toronto,1.0,Coffee Shop,Bakery,Park,Breakfast Spot,Restaurant,Café,Pub,Theater,Spa,Beer Store
3,North York,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Miscellaneous Shop,Coffee Shop,Women's Store,Vietnamese Restaurant,Electronics Store,Escape Room
4,Queen's Park,1.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Burrito Place,Bar,Italian Restaurant,Japanese Restaurant,Beer Bar,Sandwich Place,Salad Place
7,North York,1.0,Gym,Caribbean Restaurant,Japanese Restaurant,Café,Dessert Shop,Convenience Store,Discount Store,Event Space,Ethiopian Restaurant,Escape Room
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,1.0,Café,Coffee Shop,Pizza Place,Bakery,Restaurant,Pub,Italian Restaurant,Park,General Entertainment,Bank
97,Downtown Toronto,1.0,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Gym,Seafood Restaurant,Salad Place,Steakhouse,Deli / Bodega
99,Downtown Toronto,1.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Pub,Fast Food Restaurant,Yoga Studio,Hotel,Mediterranean Restaurant
100,East Toronto Business,1.0,Light Rail Station,Skate Park,Spa,Restaurant,Auto Workshop,Burrito Place,Brewery,Garden,Fast Food Restaurant,Garden Center


**Cluster 3**

In [344]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,2.0,Lawyer,Women's Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


**Cluster 4**

In [345]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3.0,Fast Food Restaurant,Park,Food & Drink Shop,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
21,York,3.0,Park,Women's Store,Pool,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
35,East York/East Toronto,3.0,Park,Coffee Shop,Convenience Store,Metro Station,Women's Store,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
40,North York,3.0,Park,Airport,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
52,North York,3.0,Park,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
61,Central Toronto,3.0,Park,Bus Line,Swim School,Women's Store,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
66,North York,3.0,Park,Electronics Store,Convenience Store,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
68,Central Toronto,3.0,Park,Trail,Sushi Restaurant,Jewelry Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
77,Etobicoke,3.0,Park,Sandwich Place,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
85,Scarborough,3.0,Park,Playground,Intersection,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


**Cluster 5**

In [346]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,4.0,Baseball Field,Food Truck,Women's Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Diner
57,North York,4.0,Baseball Field,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store,Fast Food Restaurant
101,Etobicoke,4.0,Baseball Field,Breakfast Spot,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Women's Store
